In [37]:
import sklearn
import pickle
import praw
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

reddit = praw.Reddit(client_id='8kRfdrPdpbJe9Q', client_secret='qeSfFi6kizAdurAFL790ZhQL_uM', user_agent='Anukriti Jain')
loaded_model = pickle.load(open('website/model.sav', 'rb'))

tokenizer = RegexpTokenizer(r'\w+')
stopwords_eng = set(stopwords.words('english'))
stemmer = PorterStemmer()
#d = enchant.Dict("en_US")

def getStemmedText(text):
    
    text = text.lower()
    post = text
    #post = text.split()
    #post = ""
    #for word in new_text:
    #   if d.check(word)==True:
    #        post = post + " " + word
    
    tokens = tokenizer.tokenize(post)
    new_tokens = []
    stemmed_tokens = []
    
    for token in tokens:
        if token not in stopwords_eng:
            new_tokens.append(token)
    
    for token in new_tokens:
        stemmed_tokens.append(stemmer.stem(token))
        
    cleaned_text = ' '.join(stemmed_tokens)
    return cleaned_text
    
def detect_flair(url,loaded_model):

    submission = reddit.submission(url=url)

    data = {}

    data['title'] = submission.title
    data['url'] = submission.url
    data['body'] = submission.selftext

    submission.comments.replace_more(limit=None)
    comment = ''
    count = 0
    for top_level_comment in submission.comments:
        if(count<5):
            comment = comment + ' ' + top_level_comment.body
        count+=1
    data["comment"] = comment
  
    data['title'] = getStemmedText(data['title'])
    data['comment'] = getStemmedText(data['comment'])
    data['body'] = getStemmedText(data['body'])
    data['url'] = getStemmedText(data['url'])
    
    data['combine'] = data['title'] + data['comment'] + data['url'] + data['body']
  
    return loaded_model.predict([data['combine']])

In [40]:
ans = detect_flair('https://www.reddit.com/r/india/comments/d0ukr4/as_a_normal_citizen_how_afraid_should_i_be_of_the/', loaded_model)

In [41]:
print(ans.item())

AskIndia
